In [1]:
import sys
sys.path.insert(0, '..')

from metrics.charades_classify import *

In [2]:
gt_path = '/data/Datasets/Charades/Annotations/Charades_v1_causal_clip_test.txt'
cls_file = '/data/OnlineActionRecognition/outputs/causal_eval_charades_r50_i3d_nl_32x2/causal_eval_charades_r50_i3d_nl_32x2_4gpus_full_clip_joined.txt'

In [3]:
gt_ids, gt_classes = read_file(gt_path)
test_ids, test_scores = read_file(cls_file)

n = len(gt_ids)
n_test = len(test_ids)
if n < n_test:
    # Check if its duplicate items
    test_ids, test_index_order = np.unique(test_ids, return_index=True)
    test_scores = np.array(test_scores)[test_index_order]
    
assert len(gt_classes) == len(test_scores)

1153408 1153512


True

In [4]:
with open(cls_file, 'r') as file:
    text = sorted(file.readlines())

In [6]:
unique_lines = np.unique(text)
len(text), len(unique_lines)

(1153512, 1153512)

In [ ]:
results_file = '/data/OnlineActionRecognition/outputs/causal_eval_charades_r50_i3d_nl_32x2/causal_eval_charades_r50_i3d_nl_32x2_4gpus_full_clip_joined_v1.txt'
with open(results_file, 'w') as file:
    

In [4]:
total = len(gt_classes)


In [5]:
def select_ten_clips(video_ids, video_classes):
    expanded_sample_length = int(video_ids[0].split('_')[1])
    num_frames = int(video_ids[-1].split('_')[1])
    
    sample_start_pos = np.linspace(expanded_sample_length, num_frames, 10, dtype=int)
    ids = sample_start_pos - expanded_sample_length
    
    return video_ids[ids], video_classes[ids]
    

In [6]:
def divide_per_clip(ids, classes):
    clips_ids = []
    clips_classes = []

    # Selecting 10 clips
    video_name = None
    for i, video_frame in enumerate(ids):
        name = video_frame.split('_')[0]
        if name != video_name:
            # new video! But first, save old video
            if i > 0:
                ten_clips_ids, ten_clips_classes = select_ten_clips(np.array(video_ids), np.array(video_classes))
                clips_ids.append(ten_clips_ids)
                clips_classes.append(ten_clips_classes)
            # star new one
            video_ids = []
            video_classes = []
            video_name = name
            
        video_ids.append(video_frame)
        video_classes.append(classes[i])
        
    # Append last video
    ten_clips_ids, ten_clips_classes = select_ten_clips(np.array(video_ids), np.array(video_classes))
    clips_ids.append(ten_clips_ids)
    clips_classes.append(ten_clips_classes)
        
    return clips_ids, clips_classes

In [8]:
gt_clips_ids

[array(['00607_000049', '00607_000104', '00607_000159', '00607_000214',
        '00607_000269', '00607_000324', '00607_000379', '00607_000434',
        '00607_000489', '00607_000545'], dtype='<U12'),
 array(['00T1E_000049', '00T1E_000078', '00T1E_000108', '00T1E_000138',
        '00T1E_000168', '00T1E_000197', '00T1E_000227', '00T1E_000257',
        '00T1E_000287', '00T1E_000317'], dtype='<U12'),
 array(['015XE_000089', '015XE_000188', '015XE_000287', '015XE_000386',
        '015XE_000485', '015XE_000584', '015XE_000683', '015XE_000782',
        '015XE_000881', '015XE_000981'], dtype='<U12'),
 array(['01THT_000044', '01THT_000090', '01THT_000136', '01THT_000182',
        '01THT_000228', '01THT_000274', '01THT_000320', '01THT_000366',
        '01THT_000412', '01THT_000459'], dtype='<U12'),
 array(['02DPI_000049', '02DPI_000119', '02DPI_000190', '02DPI_000261',
        '02DPI_000332', '02DPI_000403', '02DPI_000474', '02DPI_000545',
        '02DPI_000616', '02DPI_000687'], dtype='<U12'),


In [9]:
test_clips_ids

[array(['00607_000049', '00607_000104', '00607_000159', '00607_000214',
        '00607_000269', '00607_000324', '00607_000379', '00607_000434',
        '00607_000489', '00607_000545'], dtype='<U12'),
 array(['00T1E_000049', '00T1E_000078', '00T1E_000108', '00T1E_000138',
        '00T1E_000168', '00T1E_000197', '00T1E_000227', '00T1E_000257',
        '00T1E_000287', '00T1E_000317'], dtype='<U12'),
 array(['015XE_000089', '015XE_000188', '015XE_000287', '015XE_000386',
        '015XE_000485', '015XE_000584', '015XE_000683', '015XE_000782',
        '015XE_000881', '015XE_000981'], dtype='<U12'),
 array(['01THT_000044', '01THT_000090', '01THT_000136', '01THT_000182',
        '01THT_000228', '01THT_000274', '01THT_000320', '01THT_000366',
        '01THT_000412', '01THT_000459'], dtype='<U12'),
 array(['02DPI_000049', '02DPI_000119', '02DPI_000190', '02DPI_000261',
        '02DPI_000332', '02DPI_000403', '02DPI_000474', '02DPI_000545',
        '02DPI_000616', '02DPI_000687'], dtype='<U12'),


In [7]:
gt_clips_ids, gt_clips_classes = divide_per_clip(gt_ids, gt_classes)
test_clips_ids, test_clips_classes = divide_per_clip(test_ids, test_scores)

In [8]:
print(len(gt_clips_ids), len(gt_clips_classes), len(test_clips_ids), len(test_clips_classes))
print(gt_clips_ids[100].shape, gt_clips_classes[100].shape, test_clips_ids[100].shape, test_clips_classes[100].shape)

1863 1863 1863 1863
(10,) (10, 157) (10,) (10, 157)


In [11]:
gt_clips_mean = [(sum(gt_c) > 0).astype(int) for gt_c in gt_clips_classes]
gt_clips_mean[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0])

In [12]:
import torch
from torch.nn import AvgPool1d, MaxPool1d

def video_output(outputs):
    max_pool = MaxPool1d(10)
    
    outputs = torch.tensor(outputs)

    data = outputs.view(1, -1, 157).contiguous()
    data = data.permute(0, 2, 1).contiguous()

    data = max_pool(data)
    video_data = data.view(157).contiguous()

    return video_data

In [13]:
test_clip_mean = [video_output(t_c).numpy() for t_c in test_clips_classes]

In [19]:
len(gt_clips_mean), len(test_clip_mean)

(1863, 1863)

In [20]:
mAP, wAP, ap = charades_map(np.array(test_clip_mean), np.array(gt_clips_mean), None)

In [21]:
np.array(test_clip_mean).shape, np.array(gt_clips_mean).shape

((1863, 157), (1863, 157))

In [22]:
mAP, wAP, ap

(0.31687666392552655,
 0.4134873028104368,
 array([0.44347489, 0.50340905, 0.45388451, 0.30980621, 0.29446716,
        0.27863818, 0.37312566, 0.04686454, 0.56941407, 0.39537935,
        0.08861613, 0.73262257, 0.40300705, 0.1749923 , 0.45494257,
        0.57251401, 0.58980523, 0.11061253, 0.19516576, 0.23739711,
        0.47145038, 0.46828916, 0.17522457, 0.14789418, 0.02810515,
        0.35992905, 0.62624139, 0.43414941, 0.19870067, 0.24283421,
        0.27579402, 0.02083456, 0.62337088, 0.32317195, 0.20025644,
        0.19535671, 0.08456864, 0.17407475, 0.26482627, 0.20266204,
        0.37111484, 0.24180401, 0.19664006, 0.31308389, 0.11113834,
        0.01174665, 0.18054049, 0.56336406, 0.14426034, 0.31558498,
        0.14865675, 0.57392168, 0.64909926, 0.31503555, 0.25667987,
        0.25081541, 0.1873871 , 0.25218073, 0.09387764, 0.68908339,
        0.06785341, 0.61673728, 0.37057757, 0.42878723, 0.04449783,
        0.39656942, 0.08244075, 0.40975173, 0.17814504, 0.25495899,
     

In [6]:
gt_clip_path = '/data/Datasets/Charades/Annotations/Charades_v1_causal_clip_test.txt'
gt_lframe_path = '/data/Datasets/Charades/Annotations/Charades_v1_causal_lastFrame_test.txt'

gt_clip_ids, gt_clip_classes = read_file(gt_clip_path)
gt_lframe_ids, gt_lframe_classes = read_file(gt_lframe_path)

In [10]:
np.all(gt_clip_ids == gt_lframe_ids)

True

In [11]:
np.all(gt_clip_classes == gt_lframe_classes)

False